In [1]:
import geemap
import ee
import os
ee.Initialize()

In [2]:
# Choose these coordinates when for the final product

vertices = [
    [
       -78.6788057698592,
        45.68985184684981 
    ],
    [
       -78.67627376454914,
        45.68985184684981 
    ],
    [
        -78.67627376454914,
         45.692621011934015
    ],
    [
        -78.6788057698592,
        45.692621011934015
    ],
    [
        -78.6788057698592,
        45.68985184684981
    ]
]

poly = ee.Geometry.Polygon(vertices)

In [ ]:
# Choose these coordinates when for the loop test

vertices = [
    [
       -78.67816051316345,
        45.687667175820906 
    ],
    [
       -78.67750068974578,
        45.687667175820906 
    ],
    [
        -78.67750068974578,
         45.68808313765678
    ],
    [
        -78.67816051316345,
        45.68808313765678
    ],
    [
        -78.67816051316345,
        45.687667175820906
    ]
]

poly = ee.Geometry.Polygon(vertices)

In [ ]:
vertices = [
    [
      -78.68270923328974,
      45.68524774437538
    ],
    [
     -78.67440511417963,
      45.68524774437538
    ],
    [
      -78.67440511417963,
      45.69305715580808
    ],
    [
      -78.68270923328974,
      45.493210350454774
    ],
    [
      -78.41757353217413,
      45.69305715580808
    ],
    [
      -78.68270923328974,
      45.68524774437538
    ]
]
    
poly = ee.Geometry.Polygon(vertices)

In [4]:
def getBit(n):
    # Returns a GEE server-side object representing `int(2^n)`
    return ee.Number(2).pow(n).int()

def addMaskBand(image):   
    qa = image.select("QA_PIXEL")
    
    dilatedCloudBit = getBit(1)
    cirrusBit = getBit(2)
    cloudBit = getBit(3)
    cloudShadowBit = getBit(4)
    snowBit = getBit(5)
    
    # Define the mask by extracting these bits and reclassifying the pixel based on the bit's value
    mask = ee.Image(0)\
        .where(qa.bitwiseAnd(dilatedCloudBit).neq(0), 1)\
        .where(qa.bitwiseAnd(cloudBit).neq(0), 2)\
        .where(qa.bitwiseAnd(cirrusBit).neq(0), 3)\
        .where(qa.bitwiseAnd(cloudShadowBit).neq(0), 4)\
        .where(qa.bitwiseAnd(snowBit).neq(0), 5)\
        .updateMask(image.select('QA_PIXEL').mask())\
        .rename("cloud_shadow_snow_mask")
    
    # return original image with this mask added as an extra band
    return image.addBands(mask)

def maskImage(image):
    cloud_shadow_snow = image.select("cloud_shadow_snow_mask")
    return image.updateMask(cloud_shadow_snow.eq(0))

In [5]:
def calcTasseledCapIndex_Wetness(image):
    wetness = image.expression(
        "(0.1446 * TM1) + (0.1761 * TM2) + (0.3322 * TM3) + (0.3396 * TM4) - (0.6210 * TM5) - (0.4186 * TM7)", {
            "TM1" : image.select("SR_B1"),
            "TM2" : image.select("SR_B2"),
            "TM3" : image.select("SR_B3"),
            "TM4" : image.select("SR_B4"),
            "TM5" : image.select("SR_B5"),
            "TM7" : image.select("SR_B7")
    }).rename("wetness").set({'system:time_start': image.get('system:time_start')})
    return wetness

def calcTasseledCapIndex_Wetness_L8(image):
    wetness = image.expression(
        "(0.1511 * TM1) + (0.1973 * TM2) + (0.3283 * TM3) + (0.3407 * TM4) - (0.7117 * TM5) - (0.4559 * TM7)", {
            "TM1" : image.select("SR_B2"),
            "TM2" : image.select("SR_B3"),
            "TM3" : image.select("SR_B4"),
            "TM4" : image.select("SR_B5"),
            "TM5" : image.select("SR_B6"),
            "TM7" : image.select("SR_B7")
    }).rename("wetness").set({'system:time_start': image.get('system:time_start')})
    return wetness

In [6]:
filters = [
    ee.Filter.bounds(poly),
    ee.Filter.lt("CLOUD_COVER", 50)
]

l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcTasseledCapIndex_Wetness_L8)
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcTasseledCapIndex_Wetness)
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcTasseledCapIndex_Wetness)

TasseledCapIndex_Wetness = ee.ImageCollection(l5.merge(l7).merge(l8)).sort('system:time_start')

In [7]:
img = TasseledCapIndex_Wetness.toBands().int16()

In [8]:
### just in case the band names are not kept when the image is exported
### save these ones to rename them offline
bandNames = img.bandNames().getInfo()

if os.path.exists('band_names.txt'):
    os.remove('band_names.txt')

with open('band_names.txt', 'w') as file:
    for b in bandNames:
        file.write(f"{b}\n")

In [9]:
from ee.batch import Export

task = Export.image.toDrive(
    image = img,
    region = poly,
    scale = 30,
    folder = 'gee_exports',
    fileNamePrefix = "TasseledCapIndex_Wetness_TimeSeries_Final_Product",
    description = "Export TasseledCapIndex Wetness time series final product",
    crs = "EPSG:32617"
)

task.start()

In [ ]:
img.getInfo()